In [ ]:
import numpy as np
import json
import csv
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import re
from IPython.display import display
from efficient_apriori import apriori
import matplotlib.pyplot as plt
from tqdm import tqdm
import collections
from efficient_apriori import apriori
import networkx as nx

In [ ]:
with open('data/kaggle/train.json') as f:
    data = json.load(f)
with open('data/kaggle/test.json') as f:
    data_test = json.load(f)
with open('data/nationalities.csv', 'r') as f:
    reader = csv.reader(f)
    nationalities = [item.lower() for sublist in list(reader)[1:] for item in sublist]
with open('data/to_remove.csv', 'r') as f:
    reader = csv.reader(f)
    to_remove = [item.lower() for sublist in list(reader) for item in sublist]

In [ ]:
baseURL = "https://en.wikipedia.org/wiki/"

ingr_size = list(['small', 'medium', 'large', 'big', 'flat'])

ingr_parts = list(['breast','breasts', 'tender', 'tenders', 'ribs', 'shoulder', 'fillet',
                   'steak', 'tigh', 'liver', 'livers',
                   'clove', 'cloves', 'leaf', 'leaves', 'flakes', 'threads', 'paste',
                   'piece', 'pieces', 'slice', 'slices', 'crumbles', 'powder', 'blend', 'mix'])

ingr_state = list(['grilled', 'roasted', 'toasted', 'fried', 'baked', 'cooked',
              'chopped', 'cut', 'sliced', 'shredded',
              'frozen', 'refrigerated', 'fresh', 'freshly', 'organic', 'natural', 'active',
              'boneless', 'skinless', 'condensed', 'ground', 'warm', 'cold'])

ingr_color = list(['purple', 'yellow', 'blue', 'red', 'white', 'black', 'green', 'brown'])

#to_remove_step1 = ingr_size+ingr_parts+ingr_state
to_remove_step2 = ingr_color



to_remove_step1  = list(['crumbles', 'ground', 'grilled', 'roasted', 'toasted', 'fried', 'baked', 'chopped',
                         'frozen', 'refrigerated', 'boneless', 'natural', 'organic', 'fresh', 'freshly',
                         'medium', 'large','big', 'small', 'breasts', 'tender', 'ribs', 'shoulder', 'paste', 'cloves', 'slivered',
                         'condensed', 'cut', 'piece', 'pieces', 'sliced', 'cooked', 'grated'])
to_remove_step2  = list(['purple', 'yellow', 'blue', 'red', 'white', 'black', 'green', 'dried'])

In [ ]:
"""def try_find_ingredient2(ingr_list):
    ingr = '+'.join(ingr_list)
    str1 = "Went to foodDB for: "+ingr+" -> "
    try:
        url = urlopen(baseURL2+ingr)
    except:
        return ""
    res = BeautifulSoup(url.read(),"html5lib");
    title = res.find(class_= "hit-name").text.lower()
    title = re.sub(r" \(.*\)","", title)
    join = [value for value in title.split() if value in ingr_list]
    if len(join) > 0:
        title = ' '.join(join)
    print(str1+title)
    return title"""

def try_find_ingredient(ingr_list):
    ingr = '_'.join(ingr_list)
    try:
        url = urlopen(baseURL+ingr)
    except:
        return ""
    res = BeautifulSoup(url.read(),"html5lib");
    title = res.find(id="firstHeading").text.lower()
    title = re.sub(r" \(.*\)","", title) #remove (desambiguation)
    return title

In [ ]:
"""def clean_ingredient(ingr):
    ingr = ingr.split(',')[0].split('(')[0].replace('%', "").lower()
    ingredient_list = [ing for ing in ingr.split() if (ing not in nationalities and not ing.isdigit()) ]
    ingredient = try_find_ingredient(ingredient_list)
    if ingredient == "":
        ingredient_list = [word for word in ingredient_list if (word not in to_remove)]
        ingredient = try_find_ingredient(ingredient_list)
        if ingredient == "":
            ingredient = ' '.join(ingredient_list)
        else:
            print('FOUND AT 2ND SEARCH')
    return ingredient"""

def clean_ingredient(ingr):
    ingr = re.sub(r'\([^)]*\)','', ingr)
    ingr = ingr.split(',')[0].replace('%', "").lower()
    ingredient_list = [ing for ing in ingr.split() if (ing not in nationalities and not ing.isdigit()) ]
    ingredient = ""
    if ingredient == "":
        ingredient_list = [word for word in ingredient_list if (word not in to_remove)]
        ingredient = ' '.join(ingredient_list)
    return ingredient

In [ ]:
def get_ingredients_from_recipe(recipe):
    ingredients = set()
    for ingr in recipe['ingredients']:
        cleaned_ingr = clean_ingredient(ingr)
        if cleaned_ingr == "" and 'leaves' in ingr:
            cleaned_ingr = 'lettuce'
        if cleaned_ingr == "" and ingr == 'boneless, skinless chicken breast':
            cleaned_ingr = 'chicken breasts'
        if cleaned_ingr:
            ingredients.add(cleaned_ingr)
            
    return list(ingredients)

In [ ]:
def print_ingredients_from_recipe(recipe):
    table = list()
    for ingr in recipe['ingredients']:
        cleaned_ingr = clean_ingredient(ingr)
        table.append((ingr, cleaned_ingr))
    df = pd.DataFrame.from_records(table, columns=['from', 'to'])
    display(df)

In [ ]:
len(data)

In [ ]:
%%time
table = list()
for recipe in tqdm(data):
    ingredients = get_ingredients_from_recipe(recipe)
    for ingr in ingredients:
        table.append(ingr)
for recipe in tqdm(data_test):
    ingredients = get_ingredients_from_recipe(recipe)
    for ingr in ingredients:
        table.append(ingr)

In [ ]:
pd.Series(table).hist()

In [ ]:
recipes_clean = list()
for recipe in tqdm(data):
    ingredients = get_ingredients_from_recipe(recipe)
    ingredients = [ingr for ingr in ingredients if ingr in frequent_ingr]
    recipes_clean.append(tuple(ingredients))
    
for recipe in tqdm(data_test):
    ingredients = get_ingredients_from_recipe(recipe)
    ingredients = [ingr for ingr in ingredients if ingr in frequent_ingr]
    recipes_clean.append(tuple(ingredients))

### Apriori implementation

In [ ]:
%%time
itemsets, rules = apriori(recipes_clean, min_support=0.005,  min_confidence=0.4)

### What are the ingredients part of the rules ?

In [ ]:
ing = collections.Counter()
for r in rules :
    ing.update(list(r.lhs))
    ing.update(list(r.rhs))
    
labels, values = zip(*ing.most_common())

indexes = np.arange(len(labels))
width = 1

plt.bar(indexes, values, width)
#plt.xticks(indexes + width * 0.5, labels)
plt.show()

print("20 most common ingredients :")
ing.most_common(20)

### Graph of connection between raw ingredients

In [ ]:
#get raw ingredient associations
ing_asso = collections.Counter()
distinct_ingredients = set()

for recipe in recipes_clean :
    len_recipe = len(recipe)
    combinations = []
    for i in range(len_recipe - 1) :
        for j in range(i + 1, len_recipe) :
            #print(i, j)
            ing1 = recipe[i]
            ing2 = recipe[j]
            
            distinct_ingredients |= set(ing1)
            
            if ing1 < ing2 :
                combinations.append(tuple([ing1, ing2]))
            else :
                combinations.append(tuple([ing2, ing1]))
    
    ing_asso.update(combinations)
    

#plot graph
G=nx.Graph()

for asso in ing_asso.most_common(100) :
    G.add_edge(asso[0][0],asso[0][1],weight=asso[1])
    
    
elarge=[(u,v) for (u,v,d) in G.edges(data=True) if d['weight'] >100]
esmall=[(u,v) for (u,v,d) in G.edges(data=True) if d['weight'] <=100]

pos=nx.spring_layout(G, scale=15) # positions for all nodes

# nodes
nx.draw_networkx_nodes(G,pos,node_size=20)

# edges
nx.draw_networkx_edges(G,pos,edgelist=elarge,
                    width=0.5)
nx.draw_networkx_edges(G,pos,edgelist=esmall,
                    width=2,alpha=0.5,edge_color='b',style='dashed')

# labels
nx.draw_networkx_labels(G,pos,font_size=10,font_family='sans-serif')

plt.axis('off')
plt.savefig("weighted_graph.png") # save as png
plt.show() # display
